# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [45]:
!pip install langchain_community langchain_experimental langchain_openai

In [46]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [47]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


In [49]:
!pip install faiss-cpu

In [50]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [51]:
!pip install google-search-results

In [52]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

# Define a placeholder function for my_stock_price_fetcher
def my_stock_price_fetcher(query):
    """
    Placeholder function for fetching stock prices.
    This needs to be replaced with a real implementation.
    """
    # For now, it will just return a sample response.
    return "The current price of AAPL is $170.34."

todo_prompt = PromptTemplate.from_template(
    "You are a stock trading assistant. Given this investment objective: {objective}, generate a todo list with clear actions like 'Check TSLA earnings', 'Monitor market open performance', or 'Review oil stock trends'."
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="StockPriceFetcher",
        func=my_stock_price_fetcher,  # Now refers to the defined function
        description="Fetches current stock prices for given ticker symbols."
    ),
]


prefix = """You are a stock market assistant who performs one task to help achieve the following investor objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [53]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [54]:
OBJECTIVE = "Generate a daily digest for a stock market investor, including current news, major stock movements, and a todo list for potential trades."

In [55]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [56]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: To create a todo list, we need to gather information about current stock prices and major stock movements.
Action: StockPriceFetcher
Action Input: TSLA, AAPL, GOOGL
Observation: The current price of AAPL is $170.34.
Thought: We should consider adding AAPL to our todo list as it has a relatively low price compared to TSLA and GOOGL.
Action: StockPriceFetcher
Action Input: TSLA, AAPL, GOOGL
Observation: The current price of AAPL is $170.34.
Thought: We should also look at the major stock movements for these companies to determine if there are any potential trades.
Action: Search
Action Input: "TSLA stock news"
Observation: [{'title': 'Tesla (TSLA) Stock Forms Ominous ‘Death Cross’', 'link': 'https://www.tipranks.com/news/tesla-tsla-stock-forms-ominous-death-cross', 'source': 'TipRanks', 'source_logo': 'https://serpapi.com/searches/67ffa8bd4b1386ff0a850996/i

{'objective': 'Generate a daily digest for a stock market investor, including current news, major stock movements, and a todo list for potential trades.'}